### 풀이
- 이항계수 수식은 다음과 같다
  - $\binom{n}{r} = \frac {n!}{r!(n-r)!}$
- 13977(이항 계수와 쿼리)에서도 살펴봤지만, 최대 $n$ 의 팩토리얼까지 미리 구해두는 방식으로 풀 수 있다.
- 그런데, `gnCr`은 사용할 수 없다. 역원을 사용하는 특성상 $p$ 가 소수여야 하기 때문. 문제의 $p$ 인 $m$ 은 소수가 아닐 수 있다.


- 2004(조합 0의 개수) 의 풀이를 참고해보자.
  - 0의 개수라 함은 팩토리얼을 구성하는 수 중에서, 10으로 나뉘어 떨어지는 수의 개수를 세는 것과 같다.
  - 이는 곧 10의 소인수인 2와 5의 개수를 구하고, 2의 배수이면서 5의 배수인 수의 개수, 즉, 둘 중에 더 작은 개수를 구하면 된다.
  - 팩토리얼을 구할 때, 통분을 하면 그 수가 없어지는 것과 같다. 그걸 고려한 개수 공식은 다음과 같다.
    - $\binom{n}{r}$ 에 대한 소수 $p$ 의 개수 $k$
    - $k = {n \over p} - {r \over p} - {(n-r) \over p}$
    - $p$ 는 소수이다. 그리고 2와 5만 구하면 됐었다.
    - 이 때 소수 $p$ 에 대해 어떤 항도 정수가 아니지 않다는 증명: https://kin.naver.com/qna/detail.nhn?d1id=11&dirId=1113&docId=53607443

- 위의 공식을 확장해야 한다. 
  - 곱해진 모든 수를 다 세어야 하기 때문. 
  - 우선, 팩토리얼을 분해해보자.
    - $n! = n_1 \cdot n_2 \cdot n_3 \cdots n_i, (1 \leq i \leq n)$
  - 각 항을 네가지 경우로 나눠서 생각할 수 있다.
    1. 1인 경우(trivial)
    2. 소수인 경우
        - 위의 $k$ 를 그대로 사용하면 된다.
    3. 소수와 소수의 곱인 경우
        - 이것도 $k$ 안에서 모두 고려됨을 알 수 있다.
          - (증명) 몰루. 근데 직관적으로 맞긴함..
    4. 소수의 거듭제곱인 경우
        - 특수하게 처리해야 한다. 
        - 2로 예시를 든다면, $n!$ 에 대해 2의 배수를 세었다는 것은, $n!$ 에 2가 한 번 곱해진 수를 센 것과 같다.
        - 그런데, $4, 8, 16, 32, \cdots$ 와 같이 2의 거듭제곱 꼴로 되어있는 수들은 2가 여러 번 곱해진 수이다.
        - 이를 올바르게 세려면 각 거듭제곱꼴을 `지수`번 만큼 추가로 더해주어야 한다.
  - 4의 경우를 다시 표현해보자. 
    - 2의 배수를 세기 이전의 $n!$ 속에 포함된 2의 거듭제곱 꼴들의 총 개수들은 다음과 같다.
      - ($2^1$ 배수의 개수) + ($2^2$ 배수의 개수) + $\cdots$ ($2^{log_2 n}$ 배수의 개수)
    - 그런데, ($2^1$ 의 배수의 개수)를 세고나면, ($2^2$ 의 배수의 개수)는 ($2^1$ 의 배수의 개수)로 감소(reduce)된다.
    - 그리고 나머지 거듭제곱 꼴들도 동일하게 영향 받는다. 
    - 마지막 항만 $2^{{log_2 n} - 1}$ 으로 감소한다.
    - 그렇다면, 동일한 패턴으로 모든 항이 전부 소모될 때 까지 반복할 수 있다.
  - 소수 $p$ 에 대해, $n!$ 에서 $p$ 의 모든 거듭제골 꼴을 포함하는 개수를 세는 식은 다음과 같다.
    - $\displaystyle {n \over p} + {n \over p^2} + \cdots {n \over p^{log_p n}} = \sum_{i=1}^{log_p n} {n \over p^i}$

- 이제 $n!$ 의 모든 소인수들에 대해 위의 식을 적용하면 된다.
  - $n$ 까지의 모든 소수를 구해야 한다.
  - $n$ 이 적당히 작으므로, 에라토스테네스의 체를 사용하면 가장 효율적으로 구해진다.
  - 이제 $n$ 을 소인수분해한 각 소수 $P \ni p$ 에 대해, $n!$ 에 각 소수가 곱해진 개수 $c_p$ 를 구한다.
  - $c_p = \sum_{i=1}^{log_p n} {n \over p^i}$

- 이제 $\binom{n}{k}$ 를 구할 차례다.
  - 지금까지 구한 것이 $n!$ 에 대한 $p \in P$ 와, 그 $p$ 가 총 몇 번 곱해졌는지의 $c_p$ 들을 구했다.
  - 즉, $p$ 를 $c_p$ 번곱한 뒤 $M$ 으로 나눠보는 과정을 모든 $P$ 에 대해 반복하면 된다.
  - 그 과정에서 수가 한번이라도 나뉘어 떨어진다면, 그 즉시 종료해야 함에 주의.

In [ ]:
import math
def sieve(n):
  if n < 2: return set()
  n += 1
  P = [1] * (n//2)
  for i in range(3, math.isqrt(n) + 1, 2):
    if P[i//2]: P[i**2//2::i] = [0]*((n-i**2-1)//(2*i) + 1)

  return [2] + [2*i+1 for i in range(1, n//2) if P[i]]

def sol() :
  N, K, M = map(int, input().split())
  S = sieve(N)
  cnt = [0] * len(S)
  
  for i, p in enumerate(S) :
    j = p
    while j <= N :
      cnt[i] += ((N // j) - (K // j) - ((N-K) // j))
      j *= p

  ans = 1
  for i, c in enumerate(cnt) :
    for _ in range(c) :
      ans = (ans * S[i]) % M
      if ans == 0 : return print(0)
  print(ans)

sol()

In [ ]:
import math
def sieve(n):
  if n < 2: return set()
  n += 1
  P = [1] * (n//2)
  for i in range(3, math.isqrt(n) + 1, 2):
    if P[i//2]: P[i**2//2::i] = [0]*((n-i**2-1)//(2*i) + 1)

  return [2] + [2*i+1 for i in range(1, n//2) if P[i]]

def sol() :
  N, K, M = map(int, input().split())
  S = sieve(N)
  C = [0] * len(S) #C[i] = nCk에 i번째 소수가 곱해진 횟수
  
  for i, p in enumerate(S) :
    j = p
    while j <= N : #n!에 곱해진 소수끼리의 곱과 소수의 거듭제골 꼴이 곱해진 횟수 구하기
      C[i] += ((N // j) - (K // j) - ((N-K) // j))
      j *= p

  ans = 1
  for c, s in zip(C, S) :
    i = 0
    #소수들과 그걸 곱할 횟수를 알았으므로 곱해주면서 % M을 구한다.
    while ans and i < c : #단, 그 과정에서 ans가 0이 되어서(나누어떨어진다는 뜻) 더이상 곱해줄 필요가 없으면 break
      ans = (ans * s) % M
      i += 1

  print(ans)

sol()

### 시간복잡도
- 에라토스테네스 + 소수별 등장 횟수 구하기 + 이항계수 % M 구하기
- $\pi(N)$ 은 $N$ 이하의 소수의 개수, $p_i$ 는 $i$ 번째 소수, $c_{p_i}$ 는 $p_i$ 가 $\binom{N}{K}$ 안에 곱해진 횟수.
- $O(N \log \log N + \sum_{i=1}^{\pi(N)} \cdot \sum_{j=1}^{\log_{p_i} N} 1 + \sum_{i=1}^{\pi(N)} 1 \cdot c_{p_i}) \\ = O(N \log \log N + \sum_{i=1}^{\pi(N)} \sum_{j=1}^{\log_{p_i} N} + \sum_{i=1}^{\pi(N)} c_{p_i})$
  - ...손도 못대겠다. 
  - 다만 2번째 항이 가장 오래걸릴 확률이 높을 것 그래서 거의 2번째 항으로 시간복잡도를 결정할 것 같다.

- 개선을 여러가지 해봤지만 별로 소득은 없었다.
- C를 구하는 과정에서 $log_p N$ 번 반복되는 것을 이용해 `int(math.log(N, p)` 로 바꿔봤지만, 그게 더 오래걸렸다.
  - 추가로, 무한급수 공식을 이용해서 $O(1)$ 만에 구해보려는 노력도 했었는데, 포기했다.
    - 아래는 고민해봤던 방법이다
    - $a // b = c_1, a // b^2 = c_2, \cdots a // b^k = c_k$ 라고 했을 때\
    $a$ 를 $b^k$ 로 매번 나눠보는 방법 이외에, $c_1$ 을 적절히 변형해서 $c_k$ 를 구할 방법이 있는가?
    - (증명) 안된다 ㅠㅠ \
    하려고자 한 것은, 수식을 $c_i = \lfloor {a \over b^i} \rfloor$ 으로 해석한것과 같다.\
    $c_1 = \lfloor {a \over b} \rfloor$ 인데, 이것을 $// b$ 해서 $c_2$ 를 구하고자 한다고 해보자.\
    그런데 $c_2 = \lfloor {c_1 \over b} \rfloor$ 은 항상 성립하지 않는다. 따라서 안된다.
    - 그럼 수렴값 연산에 부동소숫점이 들어갈 수 밖에 없는데, 자신이 없다. 따라서 포기.
- PyPy의 common pattern optimization을 노리고 반복문의 형태를 바꿔봤지만, 그것도 소득은 없었다.
  - 도대체 그것을 어떻게 실천한다는 말인가?